In [1]:
from pathlib import Path
import torch
import json
import pandas as pd
import tqdm
import math

from aau2v.config import ModelConfig, TrainerConfig
from aau2v.dataset_center import load_dataset_center
from aau2v.trainer import PyTorchTrainer
from aau2v.model import load_model, PyTorchModel
from aau2v.util import set_seed

In [2]:
def calc_accuracy(
    model: PyTorchModel,
    num_item: int,
    test_dataset: list[tuple[int, list[int], list[int]]],
    top_k: list[int],
) -> dict[str, float]:
    model.eval()
    hit_counts = {k: 0 for k in top_k}
    seq_index = torch.LongTensor(list(map(lambda d: d[0], test_dataset)))
    context_items = torch.LongTensor(list(map(lambda d: d[1], test_dataset)))
    rec_list = model.output_rec_lists(
        seq_index=seq_index,
        item_indices=context_items,
        cand_item_indices=torch.arange(num_item),
        k=max(top_k),
    )
    for i in range(len(test_dataset)):
        for k in top_k:
            hit_counts[k] += len(set(test_dataset[i][2]) & set(rec_list[i][:k]))

    total_rec = len(test_dataset)
    results = {
        f"Accuracy@{k}": hit_count / total_rec / k
        for k, hit_count in hit_counts.items()
    }
    return results

In [3]:
run_configs = [
    {
        "name": "User2Vec (d=16)",
        "model_name": "user2vec",
        "d_model": 16,
        "epochs": 10,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "User2Vec (d=32)",
        "model_name": "user2vec",
        "d_model": 32,
        "epochs": 10,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "User2Vec (d=64)",
        "model_name": "user2vec",
        "d_model": 64,
        "epochs": 10,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "AAU2V (d=16)",
        "model_name": "aau2v",
        "d_model": 16,
        "epochs": 10,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "AAU2V (d=32)",
        "model_name": "aau2v",
        "d_model": 32,
        "epochs": 10,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "AAU2V (d=64)",
        "model_name": "aau2v",
        "d_model": 64,
        "epochs": 10,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "AAU2V w/o WT",
        "model_name": "aau2v",
        "d_model": 64,
        "epochs": 10,
        "use_weight_tying": False,
        "use_attention": True,
        "use_meta": True,
    },
    {
        "name": "AAU2V w/o AT",
        "model_name": "aau2v",
        "d_model": 64,
        "epochs": 10,
        "use_weight_tying": True,
        "use_attention": False,
        "use_meta": True,
    },
    {
        "name": "AAU2V w/o AI",
        "model_name": "aau2v",
        "d_model": 64,
        "epochs": 10,
        "use_weight_tying": True,
        "use_attention": True,
        "use_meta": False,
    },
]

In [4]:
results = {}

for run_config in run_configs:
    set_seed(0)

    results[run_config["name"]] = {"accuracy": []}
    model_config = ModelConfig(
        max_embedding_norm=1,
        d_model=run_config["d_model"],
        use_attention=run_config["use_attention"],
        use_meta=run_config["use_meta"],
        use_weight_tying=run_config["use_weight_tying"],
    )
    trainer_config = TrainerConfig(
        dataset_name="movielens",
        lr=5e-5,
        model_name=run_config["model_name"],
        epochs=run_config["epochs"],
    )

    print(model_config)
    print(trainer_config)

    dataset_center = load_dataset_center(
        dataset_name=trainer_config.dataset_name,
        window_size=model_config.window_size,
    )
    print(
        "train:",
        len(dataset_center.train_dataset),
        "valid:",
        len(dataset_center.valid_dataset),
    )
    model = load_model(
        dataset_center=dataset_center,
        trainer_config=trainer_config,
        model_config=model_config,
    )
    trainer = PyTorchTrainer(
        model=model,
        dataset_center=dataset_center,
        trainer_config=trainer_config,
    )

    def on_epoch_end(epoch: int) -> None:
        result = calc_accuracy(
            model=model,
            num_item=dataset_center.num_item,
            test_dataset=dataset_center.test_datasets["test"],
            top_k=[10, 20, 30, 40, 50],
        )
        print(f"epoch: {epoch}, result: {result}")
        results[run_config["name"]]["accuracy"].append(result)
        torch.save(model, f"model/exp-5-1/{run_config['name']}-{epoch}.pt")

    losses = trainer.fit(on_epoch_end=on_epoch_end)
    results[run_config["name"]]["loss"] = losses

    with open("result.json", "w") as f:
        json.dump(results, f)

ModelConfig(d_model=16, init_embedding_std=0.2, max_embedding_norm=1, window_size=5, negative_sample_size=5, lr=5e-05, weight_decay=0, use_weight_tying=True, use_meta=True, use_attention=True)
TrainerConfig(model_name='user2vec', dataset_name='movielens', epochs=10, batch_size=64, verbose=False, ignore_saved_model=True, save_model=True, load_dataset=False, save_dataset=False, model_dir='cache/model/', dataset_dir='cache/dataset/', device='cpu')
create dataset
num_seq: 6040, num_item: 3706, num_item_meta: 28, num_seq_meta: 30, num_item_meta_types: 3, num_seq_meta_types: 3
train: 589192 valid: 110547


100%|██████████| 9207/9207 [00:33<00:00, 278.97it/s]


train 0.006489633658498872


100%|██████████| 1728/1728 [00:02<00:00, 818.38it/s]


valid 0.006489873384934815
epoch: 0, result: {'Accuracy@10': 0.039157762413576366, 'Accuracy@20': 0.03852922690131992, 'Accuracy@30': 0.03773308191912843, 'Accuracy@40': 0.03648648648648649, 'Accuracy@50': 0.03634192331866751}
saved model to cache/model/movielens/user2vec.pt


...

valid 0.004687638868859454
epoch: 8, result: {'Accuracy@10': 0.15958516656191074, 'Accuracy@20': 0.15524827152734128, 'Accuracy@30': 0.15156086318877016, 'Accuracy@40': 0.1477372721558768, 'Accuracy@50': 0.14398491514770584}
saved model to cache/model/movielens/aau2v.pt


100%|██████████| 9207/9207 [01:04<00:00, 142.64it/s]


train 0.004519878872175085


100%|██████████| 1728/1728 [00:02<00:00, 623.32it/s]


valid 0.0046572648845658645
epoch: 9, result: {'Accuracy@10': 0.15518541797611565, 'Accuracy@20': 0.15012570710245127, 'Accuracy@30': 0.1463440184370417, 'Accuracy@40': 0.14252042740414833, 'Accuracy@50': 0.13981143934632306}
saved model to cache/model/movielens/aau2v.pt


In [10]:
top_k = list(range(10, 51, 20))
data = {}
for method, result in results.items():
    data[method] = []
    for k in top_k:
        a = max(map(lambda r: r[f"Accuracy@{k}"], result["accuracy"]))
        data[method].append(a)

In [11]:
pd.options.display.precision = 4
df = pd.DataFrame(data).T
df.columns = [f"Accuracy@{k}" for k in top_k]
df

,Accuracy@10,Accuracy@30,Accuracy@50
User2Vec (d=16),0.1804,0.1750,0.1689
User2Vec (d=32),0.1927,0.1890,0.1815
User2Vec (d=64),0.2128,0.2029,0.1921
AAU2V (d=16),0.2229,0.2070,0.1945
AAU2V (d=32),0.2258,0.2110,0.1966
AAU2V (d=64),0.2272,0.2105,0.1973
AAU2V (wo weight-tying),0.2233,0.2101,0.1988
AAU2V (wo attention),0.2255,0.2115,0.1963
AAU2V (wo meta),0.2132,0.2029,0.1918
